# 1. Чтение CSV, обработка данных и сохранение в Parquet

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Инициализация SparkSession
spark = SparkSession.builder.appName("Example").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("data.csv", header=True, inferSchema=True)

# Фильтрация данных
df_filtered = df.filter(col("age") > 30)

# Выбор нужных колонок
df_selected = df_filtered.select("name", "age", "city")

# Сохранение в формате Parquet
df_selected.write.parquet("output.parquet", mode="overwrite")

# Остановка Spark
spark.stop()

# 2. Использование RDD для обработки данных

In [1]:
from pyspark.sql import SparkSession

# Инициализация SparkSession
spark = SparkSession.builder.appName("RDDExample").getOrCreate()

# Создание RDD из списка
data = [("Alice", 30), ("Bob", 25), ("Charlie", 35)]
rdd = spark.sparkContext.parallelize(data)

# Преобразование RDD (фильтрация по возрасту)
filtered_rdd = rdd.filter(lambda x: x[1] > 28)

# Преобразование в DataFrame
df = filtered_rdd.toDF(["name", "age"])

# Вывод результата
df.show()

# Остановка Spark
spark.stop()


25/04/01 22:22:55 WARN Utils: Your hostname, MacBook-Pro-Vladimir-2.local resolves to a loopback address: 127.0.0.1; using 192.168.100.95 instead (on interface en0)
25/04/01 22:22:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 22:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+---+
|   name|age|
+-------+---+
|  Alice| 30|
|Charlie| 35|
+-------+---+



# 3. Агрегация данных в PySpark
Считаем средний возраст по городам.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = SparkSession.builder.appName("AggregationExample").getOrCreate()

data = [("Alice", 30, "New York"), ("Bob", 25, "Chicago"), ("Charlie", 35, "New York")]
df = spark.createDataFrame(data, ["name", "age", "city"])

# Группировка по городу и вычисление среднего возраста
df_grouped = df.groupBy("city").agg(avg("age").alias("average_age"))

df_grouped.show()

spark.stop()

+--------+-----------+
|    city|average_age|
+--------+-----------+
|New York|       32.5|
| Chicago|       25.0|
+--------+-----------+



# 4. Джойн двух DataFrame (SQL JOIN)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("JoinExample").getOrCreate()

data1 = [(1, "Alice"), (2, "Bob"), (3, "Charlie")]
data2 = [(1, "New York"), (2, "Chicago"), (4, "Los Angeles")]

df1 = spark.createDataFrame(data1, ["id", "name"])
df2 = spark.createDataFrame(data2, ["id", "city"])

# INNER JOIN по id
df_joined = df1.join(df2, on="id", how="inner")

df_joined.show()

spark.stop()


+---+-----+--------+
| id| name|    city|
+---+-----+--------+
|  1|Alice|New York|
|  2|  Bob| Chicago|
+---+-----+--------+



# 5. Использование оконных функций
Добавляем колонку с порядковым номером по убыванию возраста.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

spark = SparkSession.builder.appName("WindowExample").getOrCreate()

data = [("Alice", 30), ("Bob", 25), ("Charlie", 35)]
df = spark.createDataFrame(data, ["name", "age"])

# Определяем окно сортировки
window_spec = Window.orderBy(df["age"].desc())

# Добавляем порядковый номер
df = df.withColumn("rank", row_number().over(window_spec))

df.show()

spark.stop()


25/04/01 22:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/01 22:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/01 22:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+---+----+
|   name|age|rank|
+-------+---+----+
|Charlie| 35|   1|
|  Alice| 30|   2|
|    Bob| 25|   3|
+-------+---+----+

